[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/un-fao/FAO-Water-Applications/blob/main/Transhumance/Transhumance.ipynb)

In [17]:
### Import libraries
import ee
import datetime
import ipywidgets as widgets
from ipywidgets import Layout, AppLayout, Dropdown, Output, Textarea, VBox, Label, DatePicker, HBox
import pandas as pd
from pandas import date_range
import geemap
import bqplot as bq
import numpy as np
import os
import ipyleaflet
import matplotlib.pyplot as plt
from IPython.display import display, Javascript

In [18]:
##### initialize, path to shapes 
Map = geemap.Map()
ee.Initialize()
WDPA = ee.FeatureCollection("WCMC/WDPA/current/polygons")
Ia = WDPA.filter(ee.Filter.eq("IUCN_CAT", 'Ia'))
Ib = WDPA.filter(ee.Filter.eq("IUCN_CAT", 'Ib'))
II = WDPA.filter(ee.Filter.eq("IUCN_CAT", 'II'))
WDPA = Ia.merge(Ib).merge(II)
countries = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0")
MLI = countries.filter(ee.Filter.eq("ADM0_NAME", 'Mali'))
BFA = countries.filter(ee.Filter.eq("ADM0_NAME",'Burkina Faso'))
countries = MLI.merge(BFA)
Map.addLayer(countries, {}, 'Domaine');
Buffer_BF = ee.FeatureCollection("users/jeanhounkpe/Clip_Data/Buffer_15_BF")
Buffer_ML = ee.FeatureCollection("users/jeanhounkpe/Clip_Data/Buffer_15_ML")
buffer = Buffer_BF.merge(Buffer_ML)
Cattle = ee.Image("users/yira8y/da_glw_ha")
Map.centerObject(countries)

Mali = ee.FeatureCollection("users/yira8y/mli_adm3")
Protected_Area = ee.FeatureCollection("WCMC/WDPA/current/polygons")
BFA = ee.FeatureCollection("users/yira8y/BFA")
Boundary = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")
Accessibility = ee.Image("Oxford/MAP/accessibility_to_cities_2015_v1_0")
Coul_Transh_BF = ee.FeatureCollection("users/jeanhounkpe/Clip_Data/Couloirs_Transh_BF")
Coul_Transh_ML = ee.FeatureCollection("users/jeanhounkpe/Clip_Data/Couloirs_Transh_ML")
couloir = Coul_Transh_BF.merge(Coul_Transh_ML)
Marches_BF = ee.FeatureCollection("users/jeanhounkpe/Clip_Data/Marches_BF")
Marches_ML = ee.FeatureCollection("users/jeanhounkpe/Clip_Data/Marches_ML")
marche = Marches_BF.merge(Marches_ML)
Post_Veteri_BF = ee.FeatureCollection("users/jeanhounkpe/Clip_Data/Postes_veterinaires_BF1")
Post_Veteri_ML = ee.FeatureCollection("users/jeanhounkpe/Clip_Data/Postes_veterinaires_ML1")
veto= Post_Veteri_BF.merge(Post_Veteri_ML)
Quai_Embq_BF = ee.FeatureCollection("users/jeanhounkpe/Clip_Data/Quai_Embaquement_BF")
Cheptel_Max = ee.FeatureCollection("users/jeanhounkpe/Clip_Data/Effectif_Cheptel_Max_BF")
Cheptel_Mean = ee.FeatureCollection("users/jeanhounkpe/Clip_Data/Effectif_Cheptel_Mean_BF")
Cheptel_Min = ee.FeatureCollection("users/jeanhounkpe/Clip_Data/Effectif_Cheptel_Min_BF")
Road_BF = ee.FeatureCollection("users/jeanhounkpe/Clip_Data/Routes_BF")
Villages_BF = ee.FeatureCollection("users/jeanhounkpe/Clip_Data/Villages_BF")
Countries_Bounda = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0")
# Animal spatial repartition 
Cattle = ee.Image("users/jeanhounkpe/Clip_Data/6_Ct_2015_Aw2_ML_BF")
Goats = ee.Image("users/jeanhounkpe/Clip_Data/6_Gt_2015_Aw_ML_BF")
Sheep = ee.Image("users/jeanhounkpe/Clip_Data/6_Sh_2015_Aw_ML_BF")
pointEauBF=ee.FeatureCollection("users/jeanhounkpe/Points_dEau_BF")
pointEauMLI=ee.FeatureCollection("users/jeanhounkpe/Points_dEau_ML")
pointEau = pointEauBF.merge(pointEauMLI)

In [19]:
##########Widgets

#bouton
style = {'description_width': 'max-content'}

Obtressources = widgets.Button(
    description="Obtenir ressources",
    button_style='info',
    tooltip='Cliquer pour obtenir les variables',
    layout=widgets.Layout(width="auto", height="auto"),
    style=style,
)

#Téléchargeur
uploader = widgets.FileUpload(
    description='Charger zone',
    accept='.zip, .json, .geojson',
    multiple=False,
    button_style='primary',
    tooltip='Fichier Zip.',
    style=style,
)

#soumetteur
submit = widgets.Button(
    description='Afficher zone/region', 
    button_style='success',
    tooltip='Afficher la zone chargée ou la region',
    style=style
)
#reinitialiseur
reset = widgets.Button(
    description='Effacer',
    button_style='warning',
    tooltip='Effacer la zone chargée', 
    style=style
)
#taux degradation
tauxDegradation = widgets.FloatSlider(
    value=0.05,
    min=0,
    max=1,
    step=0.01,
    description='Degrad. MS:'
)
#coefficient accessibilité
coefAccessLigneux = widgets.FloatSlider(
    value=0.3,
    min=0,
    max=1,
    step=0.01,
    description='Acces_Lign.:'
)
#coefficient accessibilité
coefAccessResidusCult = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1,
    step=0.01,
    layout=widgets.Layout(width="auto", height="auto"),
    description='Acces_Cult.:'
)
#cnsommation moyenne quotidien
feedIntakeCoef = widgets.FloatText(
    value=3.5,
    layout=widgets.Layout(width="auto", height="auto"),
    description='Conso. QM:',
)

choisirVariable = widgets.Dropdown(
    options=['Couche à exporter', 'Biomasse_accessible', 'Biomasse_totale'], value='Couche à exporter', layout=widgets.Layout(width="auto", height="auto")
)

exporter = widgets.Button(
    description="Exporter couche",
    button_style='info',
    tooltip='Exporter variable',
    style=style,
)

#date_picker = widgets.DatePicker(description='Date de la requête', value=datetime.datetime(2020,10,1))
#startYear = widgets.DatePicker(description='Fin transhumance précédente', value=datetime.datetime(2020,1,1))

startDatePicker = widgets.DatePicker(
    description='Début',
    value=datetime.datetime(2021,1,1)
    )
requestDatePicker = widgets.DatePicker(
    description='Requete',
    value=datetime.datetime(2021,3,1)
    )

#Ratio Bimass sup vs Bimass sous

ratioBimassSup1 = widgets.FloatSlider(
    value=0.9,
    min=0,
    max=1,
    step=0.01,
    description='% MS aerienne:'
    )

#Effaceur
boutoneffacer = widgets.Button(
    description='Effacer zone',
    button_style='warning',
    tooltip='Effacer la zone', 
    style=style
)

boutonexportercsv = widgets.Button(
    description='Exporter CSV',
    tooltip='Expoter information sur la zone', 
    style=style
)

choisirRegion = widgets.Dropdown(
    options=['Choisir region', 'Kayes', 'Koulikouro', 'Sikasso', 'Mopti', 'Tombouctou'], value='Choisir region', layout=widgets.Layout(width="auto", height="auto")
)

In [20]:
def resources(btn):
   
    ###Define Wet and Dry seasons for the year of the query date
##### wet starts on Jun 1st and ends on September 30 of the query year
##### Dry starts on Nov 1st of previous year and ends May query year
#####get the query year

    st_time = startDatePicker.value
    end_time = requestDatePicker.value
    startYear = datetime.datetime.strftime(st_time,'%Y-%m-%d')
    queryDate = datetime.datetime.strftime(end_time,'%Y-%m-%d')

    startWet = startYear;
    endWet = queryDate;
    startDry = startYear;
    endDry = queryDate;
    
    queryDateNotString = ee.Date(queryDate);
    yearBefore = queryDateNotString.advance(-1, 'year')
    
####Clear map
    Map.clear_layers()
    Map.remove_legends()
    Map.remove_colorbars()
    Map.add_basemap(basemap='ROADMAP')
    
#Load Dynamic world image for the wet season
    dw = geemap.dynamic_world(countries, yearBefore, queryDate, return_type='class').clip(countries)
    dwVis = { "min": 0, "max": 8, "palette": ["#419BDF", "#397D49", "#88B053", "#7A87C6", "#E49635", "#DFC35A", "#C4281B", "#A59B8F", "#B39FE1",]}
##########Add LULC to Map
    Map.addLayer(countries, {}, 'Mali_Burkina',1);
    #Map.addLayer(dw, dwVis, 'DW Land Cover', False)
    #Map.add_legend(title="Dynamic World", builtin_legend='Dynamic_World')
    #Map.addLayer(WDPA, {'color':'green'},'WDPA', False )
    Map.centerObject(countries)
    #Map
    

    
##########Use WaPOR LULC Map to excude Irrigated land
    waporLulc = ee.ImageCollection('projects/fao-wapor/L1/L1_LCC_A');
    waporLulc =waporLulc.limit(1, 'system:time_start', False).first(); ### stops in 2016 therefore using the most recent recent image
    waporLulc = waporLulc.clip(countries);
    notIrrigated = waporLulc.neq(42);
    notIrrigated = notIrrigated.updateMask(notIrrigated);
    
###### Select DW LULC of interest for the dry season
    grazingAreaDry = dw.eq(1).add(dw.eq(2)).add(dw.eq(3)).add(dw.eq(5)).add(dw.eq(6)).add(dw.eq(7));
    grazingAreaDry = grazingAreaDry.updateMask(grazingAreaDry);
    
    ### Exclude irrigated land during dry season
    grazingAreaDry = grazingAreaDry.updateMask(notIrrigated);

###### Select DW LULC of interestfor the rainin season
    grazingAreaWet = dw.eq(4)
    grazingAreaWet = grazingAreaWet.updateMask(grazingAreaWet);

    grazingAreaAll = dw.eq(1).add(dw.eq(2)).add(dw.eq(3)).add(dw.eq(4)).add(dw.eq(5)).add(dw.eq(6)).add(dw.eq(7));
    grazingAreaAll = grazingAreaAll.updateMask(grazingAreaAll)
    
#Show on Map
    #Map.addLayer(grazingAreaDry, {'palette': 'green'},'Zone_VegNat', False )
    #Map.addLayer(grazingAreaWet, {'palette': 'green'},'zone_cultures', False )   
    #Map.addLayer(grazingAreaAll, {'palette': 'green'},'zoneTout', False )   
    
    #extraire valeur du % matiere sèche sup
    ratioBimassSup = ratioBimassSup1.value
# WAPOR NPP image filtered from Start Date to  date of query  
    startTransh = startYear     #when Biomass production start
    queryDate = queryDate       #when query is made
    nppPluvieuse = ee.ImageCollection('FAO/WAPOR/2/L1_NPP_D')\
            .filterDate(startTransh, queryDate)\
            .filterBounds(countries)\
            .filter(ee.Filter.calendarRange(5, 9, 'month'))
    joursPluvieuse=nppPluvieuse.size().getInfo()*10
    nppPluvieuse=nppPluvieuse.mean().divide(1000).multiply(ratioBimassSup).multiply(joursPluvieuse).clip(countries)

    nppSeche = ee.ImageCollection('FAO/WAPOR/2/L1_NPP_D')\
            .filterDate(startTransh, queryDate)\
            .filterBounds(countries)\
            .filter(ee.Filter.calendarRange(10, 4, 'month'))
    joursSeche=nppSeche.size().getInfo()*10
    nppSeche=nppSeche.mean().divide(1000).multiply(ratioBimassSup).multiply(joursSeche).clip(countries)
    
    if joursPluvieuse == 0:
        npp = nppSeche
    elif joursSeche == 0:
            npp = nppPluvieuse
    else:
        npp = nppPluvieuse.add(nppSeche)
    
    
    nppWet=npp.multiply(grazingAreaWet).rename('nppWet')
    nppDry=npp.multiply(grazingAreaDry).rename('nppDry')
    nppAll=npp.multiply(grazingAreaAll).rename('nppAll')

#Days between start and query date
    startDate = ee.Date(startTransh);
    queryDate = ee.Date(queryDate);
    numberDays = queryDate.difference(startDate, 'days');

#NPP to DMP (from gC/m2 to  kgDM/ha)
    dmpWet=nppWet.multiply(22.222).rename('dmpWet')
    dmpDry=nppDry.multiply(22.222).rename('dmpDry')
    dmpAll = nppAll.multiply(22.222).rename('dmpAll')
    dmpWetTransh = dmpWet.clip(buffer)
    dmpDryTransh = dmpDry.clip(buffer)
    dmpAllTransh = dmpAll.clip(buffer)
    
    nppTransh = npp.clip(buffer)



    visDmp = {'min': 0, 'max': 7500, 'palette': ['white','yellow', 'green']}
    visNpp = {'min': 0, 'max': 150, 'palette': ['white','yellow', 'green']}
    #Show on Map
    #Map.addLayer(dmpWet, visDmp,'Matière_Sèche_Culture', False )
   # Map.addLayer(dmpDry, visDmp,'Matière_Sèche_Veg._Naturelle' )
    Map.addLayer(dmpAll, visDmp,'Matière_sèche_totale_Produite', False )
    #Map.addLayer(dmpWetTransh, visDmp,'Mat_Sèch_Total_Culture_Parcours_kg/ha', False )
    #Map.addLayer(dmpDryTransh, visDmp,'Mat_Sèch_Total_Veg._Naturelle_Parcours_kg/ha', False )
    
    
     #use query date and 3 months before to collecte Fire Information for Resource Management System (FIRMS) dataset
    queryDate = ee.Date(queryDate);
    daysBefore = queryDate.advance(-90, 'day')
    fire = ee.ImageCollection('FIRMS').select('T21')\
            .filterDate(daysBefore, queryDate)\
            .filterBounds(countries)\
            .mean().clip(countries)
    visFire = {'min': 0, 'max': 500, 'palette': ['red', 'orange', 'yellow']};
    Map.addLayer(fire, visFire, 'Feu',False);
    #Map.add_colorbar(visFire, discrete=False, label="Fire", orientation="horizontal", position='bottomleft', layer_name="Fire", transparent_bg=True,)
    fire = fire.multiply(0) 
    fireMask1 = fire.unmask(1)
    fireMask1= fireMask1.clip(countries)
    #Map.addLayer(fireMask1, {}, 'FeuMask',False);
    
    ##################Affichage Données Betail DLW (gridded livestock world)##############
    visCattle = {'min': 0, 'max': 40, 'palette': ['white','red', 'black']}
    #Map.addLayer(Cattle, visCattle,'Betail_Tête/ha', False )
    
    ##################Consommation de Matière sèche######
    
#Daily feed intake coef (1.75 to 2 kg/day per 100 Kg of Cattle live weight -FAO 2020 )
    feedIntakeCoef1 = feedIntakeCoef.value;
    feedIntakeCoef1 = float(feedIntakeCoef1)
    feedIntake = Cattle.multiply(feedIntakeCoef1).multiply(numberDays);   ####check if coeff accessibility considered
    visCattleIntake = {'min': 0, 'max': 20000, 'palette': ['white','yellow', 'green']};
    #Map.addLayer(feedIntake, visCattleIntake,'Consommation_Mat_Sèch_kg/ha', False );   
    
    ##########Mask paturage Ligneux#############
    maskLigneux = dw.eq(1).add(dw.eq(2)).add(dw.eq(3)).add(dw.eq(5)).add(dw.eq(6)).add(dw.eq(7));
    maskLigneux = maskLigneux.updateMask(maskLigneux);
    ##########Mask Cultures###############
    maskCultures = dw.eq(4);
    maskCultures = maskCultures.updateMask(maskCultures);
    
    ##########Accessible DM kgDM/ha   production times coef. times non degraded Ligneux et culture
    #Taux de degradation de la Matière sèche (incluse en considerant que 5% de la production est perdue par degradation)
    #Widget taux degardation Matière Sèche
    tauxDegradationVal = tauxDegradation.value
    tauxDegra = 1-tauxDegradationVal
    
    #Coefficient accessibilité
    coefAccessLigneux1 = coefAccessLigneux.value         ####
    coefAccessResidusCult1 = coefAccessResidusCult.value     ####guessed figure
    
    
    ##Ligneux
    dmpWetLigneuxAcc = dmpDry.multiply(coefAccessLigneux1);
    dmpWetLigneuxAcc = dmpWetLigneuxAcc.multiply(tauxDegra);
    #dmpWetLigneuxAcc = dmpWetLigneuxAcc.updateMask(maskLigneux); # should be activated, not working for, reason = format
    #Map.addLayer(dmpWetLigneuxAcc, visDmp,'Mat_Sèch_Access_Ligneux', False )
    ##Cultures
    dmpDryCulturesAcc = dmpWet.multiply(coefAccessResidusCult1)
    dmpDryCulturesAcc = dmpDryCulturesAcc.multiply(tauxDegra);
    #dmpDryCulturesAcc = dmpDryCulturesAcc.updateMask(maskCultures);  # should be activated, not working for, reason = format
    #Map.addLayer(dmpDryCulturesAcc, visDmp,'Mat_Sèch_Access_Culture', False )
    ##Ligneux + Culture
    #dmpLigneuxCulturesAcc = dmpWetLigneuxAcc
    
    
    ############################################################################
    ####################Revoir partie ci-dessous en dissociant MS produite par cultures et Ms produite Ligneux et rendre cela calculable pour le fichier importé
    ############################################################################
    
    ##Exclude burned areas
    #dmpLigneuxCulturesAcc = dmpLigneuxCulturesAcc.multiply(fireMask1)
    dmpWetLigneuxAcc = dmpWetLigneuxAcc.multiply(fireMask1)
    dmpDryCulturesAcc = dmpDryCulturesAcc.multiply(fireMask1)
    
    #Toute la zone
    #Map.addLayer(dmpLigneuxCulturesAcc, visDmp,'Mat_Sèch_Totale_Accessible_kg/ha', False )
    Map.addLayer(dmpWetLigneuxAcc, visDmp,'Mat_Sèch_Veg.Naturelle_Accessible_kg/ha' )
    Map.addLayer(dmpDryCulturesAcc, visDmp,'Mat_Sèch_Cultures_Accessible_kg/ha')
    
    #Parcours
    #dmpLigneuxCulturesAccTransh = dmpLigneuxCulturesAcc.clip(buffer)
    #Map.addLayer(dmpLigneuxCulturesAccTransh, visDmp,'Mat_Sèch_Accessible_Parcours_kg/ha', False )
    
    
    
 
    #Bilan Matière sèche
    #feedBalance = dmpLigneuxCulturesAcc.subtract(feedIntake);
    #visBalance = {'min': -20000, 'max': 20000, 'palette': ['#d7191c','#fdae61','#ffffbf','#a6d96a','#1a9641']}
    #Map.addLayer(feedBalance, visBalance,'Bilan_Mat_Sèch_kg/ha')
      
    #Map.add_colorbar(visDmp, discrete=False, label="DMP (kg/ha)", orientation="horizontal", position='bottomleft', layer_name="nppTransh", transparent_bg=False,)
    #Map.addLayer(nppTransh, visNpp,'nppTransh', False )
    #Map.add_colorbar(visNpp, discrete=False, label="NPP (gC/m2)", orientation="horizontal", position='bottomleft', layer_name="dmpTransh", transparent_bg=True,)
    #use query date and 10 days before to collecte Sentinel image 
    queryDate = ee.Date(queryDate);
    daysBefore = queryDate.advance(-10, 'day')
    sentinelQuery = ee.ImageCollection('COPERNICUS/S2').filterDate(daysBefore,queryDate).filterBounds(countries).median().divide(10000).clip(countries)

# Calculate Modified Normalized Difference Water Index (MNDWI) using 'GREEN'(B3) and 'SWIR1' (B11)
    mndwi = sentinelQuery.normalizedDifference(['B3', 'B11']).rename(['mndwi']);
    mndwi = mndwi.gte(0);
    mndwi = mndwi.updateMask(mndwi);
    mndwi= mndwi.clip(countries)
    visMndwi = {'min': 0, 'max': 1, 'palette': ['white', 'blue'] };
    Map.addLayer(mndwi, visMndwi, 'Eau de surface',False);
#Map.add_colorbar(visMndwi, discrete=False, label="Water Index", orientation="horizontal", position='bottomleft', layer_name="Water Index", transparent_bg=True,)

#use query date and 10 days before to collecte CHIRPS Precipitation data 
    queryDate = ee.Date(queryDate);
    daysBefore = queryDate.advance(-10, 'day')
    precipitaion = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').select('precipitation')\
            .filterDate(daysBefore, queryDate)\
            .filterBounds(countries)\
            .sum().clip(countries)
    visPecipitation = {'min': 0, 'max': 200, 'palette': ['white', 'blue'] };
    Map.addLayer(precipitaion, visPecipitation, 'Precipitaion',False);
    #Map.add_colorbar(visPecipitation, discrete=False, label="Precipitation (mm)", orientation="horizontal", position='bottomleft', layer_name="Precipitation", transparent_bg=True,)
    
    # precipitation of the period
    queryDate = ee.Date(queryDate);
    start = ee.Date(startYear)
    precipitaionperiode = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').select('precipitation')\
            .filterDate(start, queryDate)\
            .filterBounds(countries)\
            .sum().clip(countries)
    
    # precipitation 5 periods precedante
    queryDate1y = queryDate.advance(-1, 'year');
    start1y = start.advance(-1, 'year');
    precipitaionperiode1y = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').select('precipitation')\
            .filterDate(start1y, queryDate1y)\
            .filterBounds(countries)\
            .sum().clip(countries)
    
    queryDate2y = queryDate.advance(-2, 'year');
    start2y = start.advance(-2, 'year');
    precipitaionperiode2y = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').select('precipitation')\
            .filterDate(start2y, queryDate2y)\
            .filterBounds(countries)\
            .sum().clip(countries)    
    
    queryDate3y = queryDate.advance(-3, 'year');
    start3y = start.advance(-3, 'year');
    precipitaionperiode3y = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').select('precipitation')\
            .filterDate(start3y, queryDate3y)\
            .filterBounds(countries)\
            .sum().clip(countries)
    
    queryDate4y = queryDate.advance(-4, 'year');
    start4y = start.advance(-4, 'year');
    precipitaionperiode4y = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').select('precipitation')\
            .filterDate(start4y, queryDate4y)\
            .filterBounds(countries)\
            .sum().clip(countries)
    
    queryDate5y = queryDate.advance(-5, 'year');
    start5y = start.advance(-5, 'year');
    precipitaionperiode5y = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').select('precipitation')\
            .filterDate(start5y, queryDate5y)\
            .filterBounds(countries)\
            .sum().clip(countries)
    
    precipitaionperiode15y= precipitaionperiode1y.add(precipitaionperiode2y).add(precipitaionperiode3y).add(precipitaionperiode4y).add(precipitaionperiode5y).divide(5)
   
    Map.addLayer(buffer, {}, 'couloir de transhumance',False);
    
    
    def get_vector(upload_widget, out_dir=None):

        import zipfile
        import glob

        if out_dir is None:
            out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)

        vector = None

        try:
            [uploaded_file] = upload_widget.value
            file = upload_widget.value[uploaded_file]
            name = file['metadata']['name']
            content = file['content']
            out_file = os.path.join(out_dir, name)
            with open(out_file, "wb") as fp:
                fp.write(content)
        
            if name.endswith('.zip'):
                with zipfile.ZipFile(out_file, "r") as zip_ref:
                    extract_dir = os.path.join(
                        out_dir, name[:-4] + "_" + geemap.random_string(3)
                    )
                    zip_ref.extractall(extract_dir)
                    files = glob.glob(extract_dir + '/*.shp')
                    if len(files) > 0:
                        shp = files[0]
                        vector = geemap.shp_to_ee(shp)
                    else:
                        files = glob.glob(extract_dir + '/*.geojson')
                        if len(files) > 0:
                            geojson = files[0]
                            vector = geemap.geojson_to_ee(geojson)
            else:
                vector = geemap.geojson_to_ee(out_file)
        
        except Exception as e:
            print(e)

        return vector
    
    def submit_clicked(b):
        if uploader._counter >= 0:
            #regionvalue= choisirRegion.value
            #zone = Mali.filter(ee.Filter.eq('admin1Name', regionvalue))
            #Map.default_style = {'cursor': 'wait'}
            #layer_name = 'Zone'
            #style = {'color': '0000ffff', 'width': 2, 'lineType': 'solid', 'fillColor': '00000000'}
            #Map.addLayer(zone.style(**style), {}, layer_name)
            #Map.centerObject(zone)
        #else:
            try:
                if uploader._counter == 0:
                    regionvalue= choisirRegion.value
                    zone = Mali.filter(ee.Filter.eq('admin1Name', regionvalue))
                else:
                    zone = get_vector(uploader)
                Map.default_style = {'cursor': 'wait'}
                layer_name = 'Zone'
                style = {'color': '0000ffff', 'width': 2, 'lineType': 'solid', 'fillColor': '00000000'}
                Map.addLayer(zone.style(**style), {}, layer_name)
                Map.centerObject(zone)

                #calulate area of the uploaded shapefile
                areaImg = ee.Image.pixelArea()
                zoneArea = areaImg.reduceRegion(
                    reducer = ee.Reducer.sum(),
                    geometry = zone,
                    scale = 1000
                )
                
                #convert area to km2 
                zoneArea = zoneArea.getInfo()
                zoneArea = zoneArea['area']
                zoneAreaKm1 = zoneArea/1000000
                zoneAreaKm2 = round(zoneAreaKm1, 2)
                
                #calculate MS All for the uploaded shapefile
                dmpLigneuxCulturesAcc1 = dmpAll.clip(zone)
                dmpLigneuxCulturesAcc2 = dmpLigneuxCulturesAcc1.reduceRegion( # sq meters \
                                reducer = ee.Reducer.sum(),
                                geometry = zone, 
                                scale = 1000,
                                maxPixels = 1e30
                              )
                dmpLigneuxCulturesAcc3 = dmpLigneuxCulturesAcc2.getInfo()
                dmpLigneuxCulturesAcc4 = dmpLigneuxCulturesAcc3['dmpAll']
                dmpLigneuxCulturesAccTon = dmpLigneuxCulturesAcc4/1000
                dmpLigneuxCulturesAccTon = round(dmpLigneuxCulturesAccTon, 2)
                
                #calculate MS Veg. Naturelle for the uploaded shapefile
                dmpWetLigneuxAcc1 = dmpWetLigneuxAcc.clip(zone)
                #Map.addLayer(dmpWetLigneuxAcc1, visDmp,'Mat_Sèch_Veg.Naturelle_Accessible_kg/ha encore' )
                dmpWetLigneuxAcc2 = dmpWetLigneuxAcc1.reduceRegion( # sq meters \
                                reducer = ee.Reducer.sum(),
                                geometry = zone, 
                                scale = 1000,
                                maxPixels = 1e30
                              )
                dmpWetLigneuxAcc3 = dmpWetLigneuxAcc2.getInfo()
                dmpWetLigneuxAcc4 = dmpWetLigneuxAcc3['dmpDry']
                dmpWetLigneuxAccTon = dmpWetLigneuxAcc4/1000
                dmpWetLigneuxAccTon = round(dmpWetLigneuxAccTon, 2)
                     
                
                
                #calculate MS Culture for the uploaded shapefile
                dmpDryCulturesAcc1 = dmpDryCulturesAcc.clip(zone)
                dmpDryCulturesAcc2 = dmpDryCulturesAcc1.reduceRegion( # sq meters \
                                reducer = ee.Reducer.sum(),
                                geometry = zone, 
                                scale = 1000,
                                maxPixels = 1e30
                              )
                dmpDryCulturesAcc3 = dmpDryCulturesAcc2.getInfo()
                dmpDryCulturesAcc4 = dmpDryCulturesAcc3['dmpWet']
                dmpDryCulturesAccTon = dmpDryCulturesAcc4/1000
                dmpDryCulturesAccTon = round(dmpDryCulturesAccTon, 2)
                     
                
                #calculate precipitation for the uploaded shapefile for the period
                precipitaionperiode1 = precipitaionperiode.clip(zone)
                precipitaionperiode2 = precipitaionperiode1.reduceRegion( 
                                reducer = ee.Reducer.mean(),
                                geometry = zone, 
                                scale = 1000,
                                maxPixels = 1e30
                              )
                precipitaionperiode3 = precipitaionperiode2.getInfo()
                precipitaionperiode4 = precipitaionperiode3['precipitation']
                precipitaionperiodemm = round(precipitaionperiode4, 2)
                
                #calculate precipitation for the uploaded shapefile for 5 periods before
                precipitaionperiode15y1 = precipitaionperiode15y.clip(zone)
                precipitaionperiode15y2 = precipitaionperiode15y1.reduceRegion( 
                                reducer = ee.Reducer.mean(),
                                geometry = zone, 
                                scale = 1000,
                                maxPixels = 1e30
                              )
                precipitaionperiode15y3 = precipitaionperiode15y2.getInfo()
                precipitaionperiode15y4 = precipitaionperiode15y3['precipitation']
                precipitaionperiode15ymm = round(precipitaionperiode15y4, 2)                
             
                
               
                #dgraph = {'MS_Total': dmpLigneuxCulturesAccTon, 'MS_Veg._Na_acces': dmpWetLigneuxAccTon, 'MS_Cul._acces': dmpDryCulturesAccTon}
                #graph=plt.bar(list(dgraph.keys()), dgraph.values(), color='g')
                

                
                #Disply shapefile values on the map
                output_widget = widgets.Output(layout={'border': '1px solid black'})
                output_control = ipyleaflet.WidgetControl(widget=output_widget, position='topright')
                Map.add_control(output_control)
                output_widget.clear_output()
                with output_widget:
                    print('Superficie de la zone: '+ str(zoneAreaKm2) + ' Km2')
                    print('MS Totale produite: '+ str(dmpLigneuxCulturesAccTon)+ ' Tonne')
                    print('MS Veg. Naturelle accessible: '+ str(dmpWetLigneuxAccTon)+ ' Tonne')
                    print('MS Culture accessible: '+ str(dmpDryCulturesAccTon)+ ' Tonne')
                    print('Pluie sur la période: '+ str(precipitaionperiodemm)+ ' mm')
                    print('Pluie cinq périodes précédentes: '+ str(precipitaionperiode15ymm)+ ' mm')
                
                statZone = {'Superficie de la zone (km2)': zoneAreaKm2, 'MS Totale produite (Tonne)': dmpLigneuxCulturesAccTon, 'MS Veg. Naturelle accessible (Tonne)': dmpWetLigneuxAccTon, 'MS Culture accessible (Tonne)': dmpDryCulturesAccTon, 'Pluie sur la periode (mm)': precipitaionperiodemm, 'Pluie cinq periodes precedentes (mm)':precipitaionperiode15ymm};
                
                def fexportCsv (btn):
                    out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
                    filename = os.path.join(out_dir, 'statistiques de la zone.csv')
                    (pd.DataFrame.from_dict(data=statZone, orient='index').to_csv(filename, header=False))
                
                boutonexportercsv.on_click(fexportCsv)
                
                def feffacer (btn):
                    output_widget.clear_output()
                    existance = Map.find_layer('Zone')
                    if existance is None:
                        Message = 'Pas de couche Zone à effacer'
                    else:
                        Map.remove_ee_layer('Zone')
                        
                def export(btn):
                    out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
                    filename = os.path.join(out_dir, choisirVariable.value+'.tif')
                    Biomasse_accessible1 = dmpLigneuxCulturesAcc1
                    Biomasse_totale1 = dmpDry
                    if choisirVariable.value == 'Biomasse_accessible':
                        geemap.ee_export_image(Biomasse_accessible1, filename=filename, scale=1000, region=zone.geometry(), file_per_band=False)
                    else:
                        if choisirVariable.value == 'Biomasse_totale':
                            geemap.ee_export_image(Biomasse_totale1, filename=filename, scale=1000, region=zone.geometry(), file_per_band=False)
                    #display(Javascript('IPython.notebook.execute_cell(2)'))

                    return
                
                exporter.on_click(export)   
                    
                uploader.value.clear()
                uploader._counter = 0
                
            except Exception as e:
                print(e)
            Map.default_style = {'cursor': 'pointer'}
            
            boutoneffacer.on_click(feffacer)


    submit.on_click(submit_clicked)
    
    
    def reset_clicked(b):
        Map.layers = Map.layers[:3]
        output_widget.clear_output()
        with output_widget:
            print('Upload shapefile or \ngeojson as a zip file')
            display(uploader)
            display(submit)
            display(reset)
        uploader.value.clear()
        uploader._counter = 0


    reset.on_click(reset_clicked)
    
    Map.addLayer(couloir, {}, 'Piste de transhumance',0);
    Map.addLayer(marche, {}, 'Marché',0);
    Map.addLayer(veto, {}, 'Poste vétérinaire',0);
    Map.addLayer(Quai_Embq_BF, {}, 'Quai embarquement',0);
    Map.addLayer(pointEau, {}, 'Point eau',0);
    
    
    
    
    return

    

Obtressources.on_click(resources)



In [21]:
WaPOR_lien = 'https://wapor.apps.fao.org/home/WAPOR_2/1'
FBS_lien = 'https://wapor.apps.fao.org/home/WAPOR_2/1'
FIRMS_lien = 'https://www.earthdata.nasa.gov/learn/find-data/near-real-time/firms/mcd14dl-nrt'
CHIRPS_lien = 'https://www.nature.com/articles/sdata201566'


lien = widgets.HTML(value = f'Sources: Matière Sèche (MS) dérivée de <a href="{WaPOR_lien}" target="_blank">WaPOR</a>, \
        Matière Sèche accessible suivant <a href="{FBS_lien}" target="_blank">FBS</a>, \
        Précipitation suivant <a href="{CHIRPS_lien}" target="_blank">CHIRPS</a>, \
        Feu selon <a href="{FIRMS_lien}" target="_blank">MCD14DL</a>.')
                    


In [22]:
AppLayout(center=Map, 
          header=HBox([VBox([startDatePicker, requestDatePicker, feedIntakeCoef]), VBox([tauxDegradation, coefAccessLigneux,coefAccessResidusCult]), VBox([ratioBimassSup1, Obtressources])]),
          left_sidebar= VBox([uploader, choisirRegion, submit,boutonexportercsv, choisirVariable, exporter, boutoneffacer]),
          footer = VBox([lien]),
          pane_widths=['15%', 1, 1],
          height='90%',
          width='100%',

          grid_gap="10px")



AppLayout(children=(HBox(children=(VBox(children=(DatePicker(value=datetime.datetime(2021, 1, 1, 0, 0), descri…